# Goal: Fairness Metrics Computation

The goal of this notebook is to computes statistical fairness metrics, after the execution of a set of mitigation techniques, given:
- a training dataset (mitigated in case of a pre-processing technique),
- a target variable,
- a sensible attribute.


# Import Libraries





In [1]:
try:
  from google.colab import drive
  drive.mount('/content/drive')
  import sys
  path_to_project = '/content/drive/MyDrive/FairAlgorithm'
  sys.path.append(path_to_project)
  !sudo apt install libcairo2-dev pkg-config python3-dev
  IN_COLAB = True
except:
  IN_COLAB = False

Mounted at /content/drive
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
pkg-config is already the newest version (0.29.2-1ubuntu3).
python3-dev is already the newest version (3.10.6-1~22.04).
python3-dev set to manually installed.
The following additional packages will be installed:
  libblkid-dev libblkid1 libcairo-script-interpreter2 libffi-dev
  libglib2.0-dev libglib2.0-dev-bin libice-dev liblzo2-2 libmount-dev
  libmount1 libpixman-1-dev libselinux1-dev libsepol-dev libsm-dev
  libxcb-render0-dev libxcb-shm0-dev
Suggested packages:
  libcairo2-doc libgirepository1.0-dev libglib2.0-doc libgdk-pixbuf2.0-bin
  | libgdk-pixbuf2.0-dev libxml2-utils libice-doc cryptsetup-bin libsm-doc
The following NEW packages will be installed:
  libblkid-dev libcairo-script-interpreter2 libcairo2-dev libffi-dev
  libglib2.0-dev libglib2.0-dev-bin libice-dev liblzo2-2 libmount-dev
  libpixman-1-dev libselinux1-dev libsepol-dev libsm-dev libxcb-render0

In [2]:
#import libraries
import numpy as np
import pandas as pd
import pickle
from sklearn.metrics import confusion_matrix

# Initialisation

In [3]:
dataset_name = "diabetes-women"  # options: [diabetes-women] TO DO: aggiungere tutte le possibilità
mitigation = "aif360-pr"  # options: [fl-cr, fl-to, aif360-rw, aif360-di, aif360-lfr, aif360-ad, aif360-pr, aif360-cepp, aif360-eopp, aif360-roc]
np.random.seed(1234)
without_model_mitigations = ['aif360-ad', 'aif360-pr'] # TODO: AGGIUNGERE I RIMANENTI
new_dataset_mitigations = ["fl-cr", "fl-to", "aif360-di", "aif360-lfr"] # TODO: AGGIUNGERE I RIMANENTI

In [4]:
# Parameters
if dataset_name == "diabetes-women":
  target_variable, target_variable_labels, sensible_attribute = 'Outcome', ['No Diabetes','Diabetes'], 'AgeCategory'
else:
  target_variable, target_variable_labels, sensible_attribute = 'A', ['0','1'], 'B'

In [5]:
# Load the correct source dataset, considering that pre-processing techniques
# modify the original dataset, while in- and post- processing do not
if mitigation in new_dataset_mitigations:
  dataset_path = path_to_project + '/data/mitigated/mitigated-{}-{}.csv'.format(dataset_name, mitigation) if IN_COLAB else '/data/mitigated/mitigated-{}-{}.csv'.format(dataset_name, mitigation)
else:
  dataset_path = path_to_project + '/data/preprocessed/preprocessed-{}.csv'.format(dataset_name) if IN_COLAB else 'data/preprocessed/preprocessed-{}.csv'.format(dataset_name)

In [6]:
df = pd.read_csv(dataset_path)
feature_cols = df.columns
sensible_values = [0, 1]  # 0 is the discriminated group, 1 the privileged one

#Retrieve Predictions and Build Confusion Matrix

Load the predictions and scores of ML algorithms after the mitigation.

In [7]:
load_path = path_to_project + '/data/predictions/predictions-{}-{}.p'.format(dataset_name, mitigation)
with open(load_path, 'rb') as fp:
    predicted_values = pickle.load(fp)

load_path = path_to_project + '/data/scores/scores-{}-{}.p'.format(dataset_name, mitigation)
try:
  with open(load_path, 'rb') as fp:
    scores = pickle.load(fp)
except FileNotFoundError:
  scores = {}
  if mitigation not in without_model_mitigations:
    for k in ['Logistic Regression', 'Decision Tree', 'Bagging', 'Random Forest', 'Extremely Randomized Trees', 'Ada Boost']:
      scores[k] = [0, 0]

We need to save the indexes of both groups `privileged` and `discriminated` in two lists.

In [8]:
sensible_indexes = df[sensible_attribute]
# save the two groups indexes
discriminated = []
privileged = []
for idx, i in enumerate(sensible_indexes):
  if i == sensible_values[0]:
    discriminated.append(sensible_indexes.index[idx])
  else:
    privileged.append(sensible_indexes.index[idx])

`y_privileged` is the part the dataset where `sensible_value` = 1 (for example `AgeCategory` = 1), and `y_discriminated` is the part of dataset where `sensible_value` = 0.

In [9]:
y_discriminated= df.loc[list(discriminated)]
y_privileged= df.loc[list(privileged)]

y_test_discriminated = y_discriminated[target_variable]
y_test_privileged = y_privileged[target_variable]

Build the confusion matrices (one for the privileged group, one for the discriminated group) for each model.


Not for in-processing that has only one ML model!

In [10]:
confusion_matrices = {}
if mitigation not in without_model_mitigations:
  for model_name in predicted_values.keys():
    temp_dict = {}
    #print(model_name)
    #print(classification_report(df[target_variable], predicted_values[model_name]))

    temporary = list()
    for idx, i in enumerate(df.index):
      temporary.append((i, predicted_values[model_name][idx]))

    y_pred_df = pd.DataFrame(temporary, columns = ['index', 'y_pred'])
    #y_pred_df = pd.DataFrame([df.index, df[target_variable], predicted_values[model_name]], columns = ['index', 'y_test', 'y_pred'])
    y_pred_df.set_index(keys=y_pred_df['index'], inplace = True)
    y_pred_df.drop(columns='index', axis=1, inplace= True)

    y_pred_discriminated = y_pred_df.loc[list(discriminated)]
    y_pred_privileged = y_pred_df.loc[list(privileged)]

    cm_discriminated = confusion_matrix(y_test_discriminated, y_pred_discriminated)
    cm_privileged = confusion_matrix(y_test_privileged, y_pred_privileged)
    temp_dict['discriminated'] = cm_discriminated
    temp_dict['privileged'] = cm_privileged
    confusion_matrices[model_name] = temp_dict
else:
    temp_dict = {}
    #print(model_name)
    #print(classification_report(df[target_variable], predicted_values[model_name]))

    temporary = list()
    for idx, i in enumerate(df.index):
      temporary.append((i, predicted_values[idx]))

    y_pred_df = pd.DataFrame(temporary, columns = ['index', 'y_pred'])
    #y_pred_df = pd.DataFrame([df.index, df[target_variable], predicted_values[model_name]], columns = ['index', 'y_test', 'y_pred'])
    y_pred_df.set_index(keys=y_pred_df['index'], inplace = True)
    y_pred_df.drop(columns='index', axis=1, inplace= True)

    y_pred_discriminated = y_pred_df.loc[list(discriminated)]
    y_pred_privileged = y_pred_df.loc[list(privileged)]

    cm_discriminated = confusion_matrix(y_test_discriminated, y_pred_discriminated)
    cm_privileged = confusion_matrix(y_test_privileged, y_pred_privileged)
    temp_dict['discriminated'] = cm_discriminated
    temp_dict['privileged'] = cm_privileged
    confusion_matrices = temp_dict


In [11]:
confusion_matrices

{'discriminated': array([[197,   3],
        [ 28,   8]]),
 'privileged': array([[196,  57],
        [ 81, 107]])}

#Functions

Terminology:

- d is the predicted value,
- Y is the actual value in the dataset
- G the protected attribute, priv= privileged group, discr=discriminated group
- L is the legittimate attribute (only for Conditional Statistical Parity)

Fairness Metrics List:

1. Group Fairness: (d=1|G=priv) = (d=1|G=discr)
2. Predictive Parity: (Y=1|d=1,G=priv) = (Y=1|d=1,G=discr)
3. Predictive Equality: (d=1|Y=0,G=priv) = (d=1|Y=0,G=discr)
4. Equal Opportunity:  (d=0|Y=1,G=priv) = (d=0|Y=1,G=discr)
5. Equalized Odds: (d=1|Y=i,G=priv) = (d=1|Y=i,G=discr), i ∈ 0,1
6. ConditionalUseAccuracyEquality: (Y=1|d=1, G=priv) = (Y=1|d=1,G=discr) and (Y=0|d=0,G=priv) = (Y=0|d=0,G=discr)
7. Overall Accuracy Equality: (d=Y, G=priv) = (d=Y, G=priv)
8. Treatment Equality: (Y=1, d=0, G=priv)/(Y=0, d=1, G=priv) = (Y=1, d=0, G=discr)/(Y=0, d=1, G=discr)
9. FOR Parity: (Y=1|d=0, G=priv) = (Y=1|d=0,G=discr)

How to evaluate the results?

Looking at the value for each corresponding metric:

- If the value is between 0 and 1-t the discriminated group suffers from unfairness
- If the value is greater than 1+t the privileged group suffers from unfairness
- If the value is between 1-t and 1+t both privileged and discriminated group have a fair treatment

t is a threshold that should be choose by the user according to the context and the goal of the task.


In [12]:
# Retrieve TP, TN, FP, FN values from a confusion matrix
def retrieve_values(cm):
  TN = cm[0][0]
  FP = cm[0][1]
  FN = cm[1][0]
  TP = cm[1][1]
  return TP, TN, FP, FN

def rescale(metric):
  metric = metric - 1
  return metric

def standardization(metric):
  if metric > 1:
    metric = 1
  elif metric < -1:
    metric = -1
  return metric

def valid(metric, th):
  if metric > 1-th and metric < 1+th:
    return True
  return False

def and_function(m1, m2, th):
  if m1 > 1+th and m2 > 1+th:
    return max(m1, m2)
  elif m1 < 1-th and m2 < 1-th:
    return min(m1, m2)
  elif valid(m1, th) and valid(m2, th):
    return max(m1, m2)
  elif (valid(m1, th) or valid(m2, th)) and (m1 > 1+th or m2 > 1+th):
    return max(m1, m2)
  elif (valid(m1, th) or valid(m2, th)) and (m1 < 1-th or m2 < 1-th):
    return min(m1, m2)
  else:
    return max(m1, m2)

In [13]:
# Fairness metrics computed using division operator
def fairness_metrics_division(confusion_matrix, discriminated, privileged, threshold = 0.15):

  TP_priv, TN_priv, FP_priv, FN_priv = retrieve_values(confusion_matrix['privileged'])
  TP_discr, TN_discr, FP_discr, FN_discr = retrieve_values(confusion_matrix['discriminated'])

  GroupFairness_discr = (TP_discr+FP_discr)/len(discriminated)
  GroupFairness_priv = (TP_priv+FP_priv)/len(privileged)
  GroupFairness = GroupFairness_discr/GroupFairness_priv

  PredictiveParity_discr = (TP_discr)/(TP_discr+FP_discr)
  PredictiveParity_priv = (TP_priv)/(TP_priv+FP_priv)
  PredictiveParity = PredictiveParity_discr/PredictiveParity_priv

  PredictiveEquality_discr = (FP_discr)/(TN_discr+FP_discr)
  PredictiveEquality_priv = (FP_priv)/(TN_priv+FP_priv)
  PredictiveEquality = PredictiveEquality_discr/PredictiveEquality_priv

  EqualOpportunity_discr = (FN_discr)/(TP_discr+FN_discr)
  EqualOpportunity_priv = (FN_priv)/(TP_priv+FN_priv)
  EqualOpportunity = EqualOpportunity_priv/EqualOpportunity_discr

  EqualizedOdds1 = ((TP_discr/(TP_discr+FN_discr)) / (TP_priv/(TP_priv+FN_priv))) # (1-equalOpportunity_discr)/(1-equalOpportunity_priv)
  EqualizedOdds2 = ((FP_discr/(TN_discr+FP_discr)) / (FP_priv/(TN_priv+FP_priv))) # = PredictiveEquality
  # EqualizedOdds = (EqualizedOdds1 * EqualizedOdds2)
  EqualizedOdds = and_function(EqualizedOdds1, EqualizedOdds2, threshold)

  ConditionalUseAccuracyEquality1 = ((TP_discr/(TP_discr+FP_discr)) / (TP_priv/(TP_priv+FP_priv)))
  ConditionalUseAccuracyEquality2 = ((TN_discr/(TN_discr+FN_discr)) / (TN_priv/(TN_priv+FN_priv)))
  # ConditionalUseAccuracyEquality = (ConditionalUseAccuracyEquality1 * ConditionalUseAccuracyEquality2)
  ConditionalUseAccuracyEquality = and_function(ConditionalUseAccuracyEquality1, ConditionalUseAccuracyEquality2, threshold)

  OAE1 = TP_discr/TP_priv
  OAE2 = TN_discr/TN_priv
  # OverallAccuracyEquality = (OAE1 * OAE2)
  OverallAccuracyEquality = and_function(OAE1, OAE2, threshold)

  try: TreatmentEquality_discr = (FN_discr/FP_discr)
  except ZeroDivisionError: TreatmentEquality_discr = 2  #max value
  try: TreatmentEquality_priv = (FN_priv/FP_priv)
  except ZeroDivisionError: TreatmentEquality_priv = 2  #max value
  TreatmentEquality = TreatmentEquality_priv/TreatmentEquality_discr

  try: FORParity_discr = (FN_discr)/(TN_discr+FN_discr)
  except ZeroDivisionError: FORParity_discr = 2  #max value
  try: FORParity_priv = (FN_priv)/(TN_priv+FN_priv)
  except ZeroDivisionError: FORParity_priv = 2  #max value
  FORParity = FORParity_priv/FORParity_discr

  FN_P_discr = (FN_discr)/len(discriminated)
  FN_P_priv = (FN_priv)/len(privileged)

  FP_P_discr = (FP_discr)/len(discriminated)
  FP_P_priv = (FP_priv)/len(privileged)

  #RecallParity = (TP_discr/(TP_discr+FN_discr))/(TP_priv/(TP_priv+FN_priv))

  metrics = {}
  metrics['GroupFairness'] = [GroupFairness, GroupFairness_discr, GroupFairness_priv]
  metrics['PredictiveParity'] = [PredictiveParity, PredictiveParity_discr, PredictiveParity_priv]
  metrics['PredictiveEquality'] = [PredictiveEquality, PredictiveEquality_discr, PredictiveEquality_priv]
  metrics['EqualOpportunity'] = [EqualOpportunity, EqualOpportunity_discr, EqualOpportunity_priv]
  metrics['EqualizedOdds'] = [EqualizedOdds, EqualizedOdds1, EqualizedOdds2]
  metrics['ConditionalUseAccuracyEquality'] = [ConditionalUseAccuracyEquality, ConditionalUseAccuracyEquality1 , ConditionalUseAccuracyEquality2]
  metrics['OverallAccuracyEquality'] = [OverallAccuracyEquality, OAE1, OAE2]
  metrics['TreatmentEquality'] = [TreatmentEquality, TreatmentEquality_discr, TreatmentEquality_priv]
  metrics['FORParity'] = [FORParity, FORParity_discr, FORParity_priv]
  metrics['FN'] = [FN_P_priv/FN_P_discr, FN_P_discr, FN_P_priv]
  metrics['FP'] = [FP_P_discr/FP_P_priv, FP_P_discr, FP_P_priv]

  for k in metrics.keys():
    value = standardization(rescale(metrics[k][0]))
    discr = metrics[k][1]
    priv = metrics[k][2]
    metrics[k] = {'Value': value, 'Discr_group': discr, 'Priv_group': priv}

  return metrics


# Fairness metrics computed using subtraction operator
def fairness_metrics_subtraction(confusion_matrix, discriminated, privileged, threshold = 0.15):

  TP_priv, TN_priv, FP_priv, FN_priv = retrieve_values(confusion_matrix['privileged'])
  TP_discr, TN_discr, FP_discr, FN_discr = retrieve_values(confusion_matrix['discriminated'])

  GroupFairness_discr = (TP_discr+FP_discr)/len(discriminated)
  GroupFairness_priv = (TP_priv+FP_priv)/len(privileged)
  GroupFairness = GroupFairness_priv-GroupFairness_discr

  PredictiveParity_discr = (TP_discr)/(TP_discr+FP_discr)
  PredictiveParity_priv = (TP_priv)/(TP_priv+FP_priv)
  PredictiveParity = PredictiveParity_priv-PredictiveParity_discr

  PredictiveEquality_discr = (FP_discr)/(TN_discr+FP_discr)
  PredictiveEquality_priv = (FP_priv)/(TN_priv+FP_priv)
  PredictiveEquality = PredictiveEquality_priv-PredictiveEquality_discr

  EqualOpportunity_discr = (FN_discr)/(TP_discr+FN_discr)
  EqualOpportunity_priv = (FN_priv)/(TP_priv+FN_priv)
  EqualOpportunity = EqualOpportunity_priv-EqualOpportunity_discr

  EqualizedOdds1 = (TP_priv/(TP_priv+FN_priv))-(TP_discr/(TP_discr+FN_discr)) # (1-equalOpportunity_discr)/(1-equalOpportunity_priv)
  EqualizedOdds2 = (FP_priv/(TN_priv+FP_priv))-(FP_discr/(TN_discr+FP_discr)) # = PredictiveEquality
  EqualizedOdds = and_function(EqualizedOdds1, EqualizedOdds2, threshold)

  ConditionalUseAccuracyEquality1 = (TP_priv/(TP_priv+FP_priv)) - (TP_discr/(TP_discr+FP_discr))
  ConditionalUseAccuracyEquality2 = (TN_priv/(TN_priv+FN_priv)) - (TN_discr/(TN_discr+FN_discr))
  ConditionalUseAccuracyEquality = and_function(ConditionalUseAccuracyEquality1, ConditionalUseAccuracyEquality2, threshold)

  OAE1 = TP_priv-TP_discr
  OAE2 = TN_priv-TN_discr
  OverallAccuracyEquality = and_function(OAE1, OAE2, threshold)

  try: TreatmentEquality_discr = (FN_discr/FP_discr)
  except ZeroDivisionError: TreatmentEquality_discr = 2  #max value
  try: TreatmentEquality_priv = (FN_priv/FP_priv)
  except ZeroDivisionError: TreatmentEquality_priv = 2  #max value
  TreatmentEquality = TreatmentEquality_priv-TreatmentEquality_discr

  try: FORParity_discr = (FN_discr)/(TN_discr+FN_discr)
  except ZeroDivisionError: FORParity_discr = 2  #max value
  try: FORParity_priv = (FN_priv)/(TN_priv+FN_priv)
  except ZeroDivisionError: FORParity_priv = 2  #max value
  FORParity = FORParity_priv-FORParity_discr

  FN_P_discr =  (FN_discr)/len(discriminated)
  FN_P_priv =  (FN_priv)/len(privileged)

  FP_P_discr = (FP_discr)/len(discriminated)
  FP_P_priv =  (FP_priv)/len(privileged)

  #RecallParity = (TP_discr/(TP_discr+FN_discr))/(TP_priv/(TP_priv+FN_priv))

  metrics = {}
  metrics['GroupFairness'] = [GroupFairness, GroupFairness_discr, GroupFairness_priv]
  metrics['PredictiveParity'] = [PredictiveParity, PredictiveParity_discr, PredictiveParity_priv]
  metrics['PredictiveEquality'] = [PredictiveEquality, PredictiveEquality_discr, PredictiveEquality_priv]
  metrics['EqualOpportunity'] = [EqualOpportunity, EqualOpportunity_discr, EqualOpportunity_priv]
  metrics['EqualizedOdds'] = [EqualizedOdds, EqualizedOdds1, EqualizedOdds2]
  metrics['ConditionalUseAccuracyEquality'] = [ConditionalUseAccuracyEquality, ConditionalUseAccuracyEquality1 , ConditionalUseAccuracyEquality2]
  metrics['OverallAccuracyEquality'] = [OverallAccuracyEquality, OAE1, OAE2]
  metrics['TreatmentEquality'] = [TreatmentEquality, TreatmentEquality_discr, TreatmentEquality_priv]
  metrics['FORParity'] = [FORParity, FORParity_discr, FORParity_priv]
  metrics['FN'] = [FN_P_priv-FN_P_discr, FN_P_discr, FN_P_priv]
  metrics['FP'] = [FP_P_discr-FP_P_priv, FP_P_discr, FP_P_priv]

  for k in metrics.keys():
    value = standardization(metrics[k][0])
    discr = metrics[k][1]
    priv = metrics[k][2]
    metrics[k] = {'Value': value, 'Discr_group': discr, 'Priv_group': priv}

  return metrics

# Print

In [14]:
if mitigation not in without_model_mitigations:
  for model_name in scores.keys():
    print(model_name)
    print("Mean Acc.", scores[model_name][0], "+/-", scores[model_name][1])
    print("-----------------------------")

In [15]:
family = ['division', 'subtraction']
models = ['Logistic Regression', 'Decision Tree', 'Bagging', 'Random Forest', 'Extremely Randomized Trees', 'Ada Boost']
metrics = ['GroupFairness', 'PredictiveParity', 'PredictiveEquality', 'EqualOpportunity', 'EqualizedOdds', 'ConditionalUseAccuracyEquality', 'OverallAccuracyEquality', 'TreatmentEquality', 'FORParity', 'FN', 'FP']

In [16]:
# Compute all the fairness metrics for all models, both with division and subtraction
div_metrics = {}
sub_metrics = {}

if mitigation not in without_model_mitigations:
  for model_name in predicted_values.keys():
    div_dict = fairness_metrics_division(confusion_matrices[model_name], discriminated, privileged)
    div_metrics[model_name] = div_dict

    sub_dict = fairness_metrics_subtraction(confusion_matrices[model_name], discriminated, privileged)
    sub_metrics[model_name] = sub_dict
else:
    div_metrics = fairness_metrics_division(confusion_matrices, discriminated, privileged)
    sub_metrics = fairness_metrics_subtraction(confusion_matrices, discriminated, privileged)

overall_metrics = {}
overall_metrics['division'] = div_metrics
overall_metrics['subtraction'] = sub_metrics

In [18]:
#Save the metrics results
save_path = path_to_project + '/measurements/metrics-{}-{}.p'.format(dataset_name, mitigation)
with open(save_path, 'wb') as fp:
    pickle.dump(overall_metrics, fp, protocol=pickle.HIGHEST_PROTOCOL)

In [17]:
#TO DO: Per confronto tenere fino a nuova funzione di stampa, poi cancellare
if mitigation not in without_model_mitigations:
  for model_name in predicted_values.keys():
    print(model_name, "\n")
    print("Fairness metrics by division")
    print(fairness_metrics_division(confusion_matrices[model_name], discriminated, privileged))
    print("\n Fairness metrics by subtraction")
    print(fairness_metrics_subtraction(confusion_matrices[model_name], discriminated, privileged))
    print("\n ----------------------------- \n")
else:
    print("Fairness metrics by division")
    print(fairness_metrics_division(confusion_matrices, discriminated, privileged))
    print("\n Fairness metrics by subtraction")
    print(fairness_metrics_subtraction(confusion_matrices, discriminated, privileged))
    print("\n ----------------------------- \n")

Fairness metrics by division
{'GroupFairness': {'Value': -0.8746641174038858, 'Discr_group': 0.046610169491525424, 'Priv_group': 0.37188208616780044}, 'PredictiveParity': {'Value': 0.11469838572642299, 'Discr_group': 0.7272727272727273, 'Priv_group': 0.6524390243902439}, 'PredictiveEquality': {'Value': -0.9334210526315789, 'Discr_group': 0.015, 'Priv_group': 0.22529644268774704}, 'EqualOpportunity': {'Value': -0.44604863221884505, 'Discr_group': 0.7777777777777778, 'Priv_group': 0.4308510638297872}, 'EqualizedOdds': {'Value': -0.9334210526315789, 'Discr_group': 0.39044652128764273, 'Priv_group': 0.06657894736842104}, 'ConditionalUseAccuracyEquality': {'Value': 0.2373922902494332, 'Discr_group': 1.114698385726423, 'Priv_group': 1.2373922902494332}, 'OverallAccuracyEquality': {'Value': -0.9252336448598131, 'Discr_group': 0.07476635514018691, 'Priv_group': 1.0051020408163265}, 'TreatmentEquality': {'Value': -0.8477443609022557, 'Discr_group': 9.333333333333334, 'Priv_group': 1.42105263157

Print example of metrics for a given model, e.g., Logistic Regression.

In [102]:
model_to_print = "Logistic Regression"
round_value = 5

print("Division \n")
for m in metrics:
  print(m, np.round(overall_metrics["division"][model_to_print][m]["Value"], round_value))
print("\nSubtraction \n")
for m in metrics:
  print(m, np.round(overall_metrics["subtraction"][model_to_print][m]["Value"], round_value))

Division 

GroupFairness 0.03943
PredictiveParity 0.0
PredictiveEquality nan
EqualOpportunity -1.0
EqualizedOdds -0.01111
ConditionalUseAccuracyEquality 0.0
OverallAccuracyEquality -0.48043
TreatmentEquality nan
FORParity -1.0
FN -1.0
FP nan

Subtraction 

GroupFairness -0.01431
PredictiveParity 0.0
PredictiveEquality 0.0
EqualOpportunity -0.01111
EqualizedOdds 0.0
ConditionalUseAccuracyEquality 0.0
OverallAccuracyEquality 1
TreatmentEquality nan
FORParity -0.0068
FN -0.00424
FP 0.0
